In [1]:
import numpy as np

In [2]:
def show_alignment(a, b, layers, gap_open, gap_continue, mismatch, match):
    s1 = ""
    s2 = ""
    i, j = len(a), len(b)
    k = np.argmax([layers[0][i, j], layers[1][i, j], layers[2][i, j]])
    
    while (i != 0) & (j != 0):
        if k == 0:
            s1, s2 = s1 + a[i - 1], s2 + "_"
            i -= 1
            if layers[k][i, j] == layers[1][i - 1, j] + gap_open + gap_continue:
                k = 1
                
        elif k == 1:
            if (a[i - 1] == b[j - 1] and layers[k][i, j] == layers[k][i - 1, j - 1] + match) or \
               (a[i - 1] != b[j - 1] and layers[k][i, j] == layers[k][i - 1, j - 1] + mismatch):
                    s1, s2 = s1 + a[i - 1], s2 + b[j - 1]
                    i, j = i - 1, j - 1
            elif layers[k][i, j] == layers[0][i, j]:
                k = 0
            else:
                k = 2
                
        else:
            s1, s2 = s1 + "_", s2 + b[j - 1]
            j -= 1
            if layers[k][i, j] == layers[1][i, j - 1] + gap_open + gap_continue: 
                k = 1
                
    if (i == 0):
        s1 += "_" * j
        s2 += (b[:j])[::-1]
        
    elif (j == 0):
        s1 += (a[:i])[::-1]
        s2 += "_" * i
        
    print(s1[::-1])
    print(s2[::-1])

In [3]:
def alignmentAffineGap(a, b, gap_open=0, gap_continue=-1, mismatch=-1, match=1):
    layer1 = np.full((len(a) + 1, len(b) + 1), gap_open)
    layer2 = np.zeros((len(a) + 1, len(b) + 1))
    layer3 = np.full((len(a) + 1, len(b) + 1), gap_open)
    
    for i in range(len(a)):
        layer1[i + 1, 0] = layer1[i, 0] + gap_continue
        
    for j in range(len(b)):
        layer3[0, j + 1] = layer3[0, j] + gap_continue
        
    for i in range(1, len(a) + 1):
        for j in range(1, len(b) + 1):
            eps = match if a[i - 1] == b[j - 1] else mismatch
            
            layer1[i, j] = max(layer1[i - 1, j] + gap_continue,
                               layer2[i - 1, j] + gap_open + gap_continue) 
            layer3[i, j] = max(layer3[i, j - 1] + gap_continue,
                               layer2[i, j - 1] + gap_open + gap_continue)
            layer2[i, j] = max(layer1[i, j], layer3[i, j],
                               layer2[i - 1, j - 1] + eps)
    
    show_alignment(a, b, [layer1, layer2, layer3], 
                   gap_open, gap_continue, mismatch, match)
    
    return [layer1, layer2, layer3]

In [4]:
def example_test():
    alignmentAffineGap("GATTACA", "GCATGCU")

In [5]:
example_test()

G_ATTACA
GCA_TGCU


In [6]:
def test1():
    alignmentAffineGap("TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC",
                      "AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC")
    
def test2():
    alignmentAffineGap("TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC",
                      "AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC", gap_open=-100, gap_continue=-0.01)
    
def test3():
    alignmentAffineGap("TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC",
                      "AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC", gap_open=0.5, gap_continue=-0.3)

In [7]:
test1()
print()
test2()
print()
test3()

___T_CC_CAGT__TATGTCAGGGGACACGAGCATG_CAGAGAC
AATTGCCGCCGTCGT_TTTCA___G_CA_G_TTATGTCAGA_TC

TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC
AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC

__T__C__C___CAGTTATGTCAGGGGACACG__A_GCATGCAGA_GAC
AATTGCCGCCGTC_GTT_T_TCA___G_CA_GTTATG__T_CAGAT__C
